<a href="https://colab.research.google.com/github/kridtapon/Gradient-Trend-Trader/blob/main/Gradient_Trend_Trader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.4 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Moving Average Slope
def calculate_ma_slope(df, period=50):
    """
    Calculate the first-order derivative (slope) of a moving average.
    """
    ma = df['Close'].rolling(window=period).mean()
    slope = ma.diff()  # First-order difference
    return slope

# Define stock symbol and period
symbol = 'META'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate indicators
df['MA_Slope'] = calculate_ma_slope(df, period=50)

# Define trend strength signals
df['Strong_Trend'] = df['MA_Slope'] > df['MA_Slope'].rolling(10).mean()
df['Weak_Trend'] = df['MA_Slope'] < df['MA_Slope'].rolling(10).mean()

# Define Entry and Exit signals
df['Entry'] = df['Strong_Trend']
df['Exit'] = df['Weak_Trend']

# Filter data for the test period
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()

[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)


Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                            84324.244376
Total Return [%]                       -15.675756
Benchmark Return [%]                   180.172856
Max Gross Exposure [%]                      100.0
Total Fees Paid                      21125.197857
Max Drawdown [%]                        64.888737
Max Drawdown Duration                      1052.0
Total Trades                                  126
Total Closed Trades                           126
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            42.063492
Best Trade [%]                          18.806411
Worst Trade [%]                        -24.277281
Avg Winning Trade [%]                    4.346387
Avg Losing Trade [%]                    -3.121303
